In [1]:
import pandas as pd
import numpy as np
import os
from collections import Counter
import nltk
import warnings
warnings.filterwarnings("ignore")

In [2]:
full_data = pd.read_csv('Full+data.csv')

# Remove some columns in full+data
x = full_data.columns[14:256]
full_data = full_data.drop(x,axis=1)
# full_data = full_data.drop(['created_at','updated_at','deleted_at'],axis=1)
# Clean/organize values in these columns
full_data['product_id'] = full_data['product_id'].str.upper()
full_data['brand'] = full_data['brand'].str.title()
full_data['name'] = full_data['name'].str.title()
full_data['description'] = full_data['description'].str.title()
full_data['brand_category'] = full_data['brand_category'].str.title()
full_data['details'] = full_data['details'].str.title()
full_data['brand_canonical_url'] = full_data['brand_canonical_url'].str.lower()

full_data.shape

(42373, 14)

In [3]:
extra = pd.read_csv('extra_data.csv')
extra['product_id'] = extra['product_id'].str.upper()
extra['brand'] = extra['brand'].str.title()
extra['name'] = extra['name'].str.title()
extra['description'] = extra['description'].str.title()
extra['brand_category'] = extra['brand_category'].str.title()
extra['details'] = extra['details'].str.title()
extra['notes'] = extra['notes'].str.lower()
extra['brand_canonical_url'] = extra['brand_canonical_url'].str.lower()
print(extra.shape)
extra.head()

(6621, 14)


,product_id,mpn,brand,brand_category,name,labels,updated_at,bc_product_id,saleable,details,created_at,brand_canonical_url,notes,description
0,01E5ZXP5H0BTEZT9QD2HRZJ47A,5529544,A.L.C.,Unknown,Lennox High Waist Cotton & Linen Pants,[],2020-04-17 15:44:57.785000+00:00,5021,2020-04-17 15:44:57.785000+00:00,"True To Size. High Rise.\n31"" Inseam; 14"" Leg ...",2020-04-15 21:59:56.695000+00:00,https://shop.nordstrom.com/s/a-l-c-lennox-high...,NaN,High-Rise Trousers Tailored From A Cool Italia...
1,01E607BHRQAJDZ76MJFN7RPRK1,5450059,Simon Miller,Unknown,Rost Belted Shorts,[],2020-04-17 16:12:25.816000+00:00,5608,2020-04-17 16:12:25.816000+00:00,"True To Size. Xs=0-2, S=4-6, M=6-8, L=8-10, Xl...",2020-04-16 00:48:54.541000+00:00,https://shop.nordstrom.com/s/simon-miller-rost...,NaN,Cinched At The Natural Waist And Pleated For F...
2,01E5ZXJ6G03R7177X723CT04W0,5526479,A.L.C.,Unknown,Minelli Silk Sleeveless Top,[{'value': 'Unsure'}],2020-04-17 15:48:01.939000+00:00,5014,NaN,"True To Size.\n25 1/2"" Length (Size Medium)\nF...",2020-04-15 21:57:46.617000+00:00,https://shop.nordstrom.com/s/a-l-c-minelli-sil...,*stand collar neckline,Painterly Brushes Of Color That Convey The Flu...
3,01E6074PQA697JZ1SBM6NM8TBG,5450071,Simon Miller,Unknown,Nepa Mismatched Button Rib Cardigan,[],2020-04-17 16:15:59.725000+00:00,5595,2020-04-17 16:15:59.725000+00:00,"True To Size. Xs=0-2, S=4-6, M=6-8, L=8-10, Xl...",2020-04-16 00:45:10.242000+00:00,https://shop.nordstrom.com/s/simon-miller-nepa...,NaN,The West Coast–Based Label Channels Beachy Vib...
4,01E606A8ARCBXW8W94DJKKNQ0D,5563429,Stateside,Unknown,Side Twist Tank Dress,[],2020-04-17 15:56:47.028000+00:00,5541,2020-04-17 15:56:47.028000+00:00,"True To Size. Xs=00-0, S=2-4, M=6-8, L=10-12.\...",2020-04-16 00:30:43.537000+00:00,https://shop.nordstrom.com/s/stateside-side-tw...,NaN,Soft Sophistication Is The Name Of The Game In...


In [4]:
attr = pd.read_excel('USC+Product+Attribute+Data+03302020.xlsx', sheet_name = 'Data')
attr['attribute_name'] = attr['attribute_name'].str.lower()
attr['attribute_value'] = attr['attribute_value'].str.lower()

attr.head()

,product_id,product_color_id,attribute_name,attribute_value
0,01DVBTBPHR8WJTCVEN5AJRHF47,01DVBTBPJ41VVT00JJCG8TTZ2W,gender,women
1,01DVA7QRXM928ZM0WWR7HFNTC1,01DVA7QRXXR9F0TWVE1HMC5ZQ3,primary color,blacks
2,01DPGV4YRP3Z8J85DASGZ1Y99W,01DPGVGBK6YGNYGNF2S6FSH02T,style,casual
3,01E1JM43NQ3H17PB22EV3074NX,01E1JM5WFWWCCCH3JTTTCYQCEQ,style,modern
4,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,01DSE8ZG8Y3FR8KWE2TY1QDWBF,shoe_width,medium


In [5]:
labels = attr.pivot_table(values='attribute_value', 
                          index=attr.product_id, 
                          columns='attribute_name', 
                          aggfunc='first',
                          dropna = False).reset_index()
labels = labels.add_prefix('attribute_')
labels

attribute_name,attribute_product_id,attribute_additional color,attribute_calf_width,attribute_category,attribute_class_blazers_coats_and_jackets,attribute_class_booties,attribute_class_boots,attribute_class_dress,attribute_class_flats,attribute_class_handbags,...,attribute_subcategory_bottom,attribute_subcategory_one_piece,attribute_subcategory_shoe,attribute_subcategory_sweater,attribute_subcategory_sweatshirt_and_hoodie,attribute_subcategory_top,attribute_toe_style,attribute_trend,attribute_upper_material,attribute_wash
0,01DMBRYVA2P5H24WK0HTK4R0A1,NaN,NaN,bottom,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01DMBRYVA2PEPWFTT7RMP5AA1T,NaN,NaN,"blazers, coats & jackets",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01DMBRYVA2Q2ST7MNYR6EEY4TK,NaN,NaN,one piece,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01DMBRYVA2S5T9W793F4CY41HE,NaN,NaN,accessory,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01DMBRYVA2ZFDYRYY5TRQZJTBD,NaN,NaN,shoe,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921,01E2M4B1MS13817G2YZA3J8ZD6,NaN,NaN,sweater,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,pullover,NaN,NaN,NaN,NaN,NaN,NaN
922,01E2M4BTQCZMT0WRCAMT9CX3PY,NaN,NaN,bottom,NaN,NaN,NaN,NaN,NaN,NaN,...,pants & leggings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
923,01E2M4E8HZ58197FV7WZ4DX80K,browns,NaN,sweater,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,pullover,NaN,NaN,NaN,NaN,NaN,NaN
924,01E2M4KKENFX66KPTHA8PRMYC3,pinks,NaN,sweatshirt & hoodie,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,pullover,NaN,NaN,NaN,NaN,NaN


In [6]:
unique_prod_id = list(set(full_data['product_id'])) + list(set(extra['product_id']))
len(unique_prod_id)

48994

In [7]:
# Select related columns for EDA

data1 = full_data[['product_id','brand','name','description','brand_category','details']] #,'details'
print(data1.columns)
print(data1.shape)
data1.tail()

Index(['product_id', 'brand', 'name', 'description', 'brand_category',
       'details'],
      dtype='object')
(42373, 6)


,product_id,brand,name,description,brand_category,details
42368,01DT5110V2ME7BY3JPCAJ91QRW,Mara Hoffman,Atlas Oversized Belted Mélange Wool Coat,Mélange Beige And Cream Wool Button Fastenings...,Clothing / Coats / Long,"Fits True To Size, Take Your Normal Size \nDes..."
42369,01DV6M2FXMPW9RSZWSXW1EK75W,Philosophy Di Lorenzo Serafini,Cropped Crochet-Trimmed Georgette Top,"Cream Georgette Ties At Neck, Concealed Hook F...",Clothing / Tops / Blouses,"Fits True To Size, Take Your Normal Size \nInt..."
42370,01DV71D0XMBM4VVJK54F2HD3ZG,Vanessa Bruno,Juna Cotton-Corduroy Mini Skirt,Sand Cotton-Corduroy Concealed Hook And Zip Fa...,Clothing / Skirts / Mini,"Fits True To Size, Take Your Normal Size \nTho..."
42371,01DV72R28K2A1AN5G167S6QRWF,Eve Denim,Annabel Rigid Mid-Rise Skinny Jean,Although Mom Jeans And Boyfriend Jeans Are All...,Women:Clothing:Jeans,Button And Zip Fastening \nComposition: 98% Co...
42372,01DSNZTZNH0NG60JWJ07KPQ465,Fila,Kessler Colorblock Track Jacket,Only At Saks. Ultra-Sporty Nylon Track Jacket ...,Themensstore/Apparel/Outerwear/Lightweightjackets,Attached Drawstring Hood\nStand Collar\nLong S...


In [8]:
data2 = extra[['product_id','brand','name','description','brand_category','details']]
print(data2.columns)
print(data2.shape)
data2.tail()

Index(['product_id', 'brand', 'name', 'description', 'brand_category',
       'details'],
      dtype='object')
(6621, 6)


,product_id,brand,name,description,brand_category,details
6616,01E2KZ045BCT1Q2Z2PBC0CWRGH,Atm Anthony Thomas Melillo,Micro Twill Pull On Pants,"These Casual Trousers Are Cut For An Easy, Dra...",Unknown,"True To Size. Xs=0, S=2-4, M=6-8, L=10. High R..."
6617,01E2KXZ0WNQ96S64TDZE3WB7CX,Atm Anthony Thomas Melillo,Brushed Twill Crop Wide Leg Pants,Brushed Twill Gives These Cropped Wide-Leg Pan...,Unknown,True To Size. High Rise.
6618,01E2KXWWY6JAGENED7P62090NP,Atm Anthony Thomas Melillo,Slim Crop Pants,"Tailored From A Soft, Dense Knit With Cropped,...",Unknown,Mid Rise. True To Size.
6619,01E2KZTEPE7QK1V8THASF7HWNR,Atm Anthony Thomas Melillo,Camo Print Silk Skirt,Flatlock-Stitched Bias Seams Enhance The Fluid...,Unknown,"True To Size. Xs=0, S=2-4, M=6-8, L=10."
6620,01E2KY2T6YV7R4VXT4BVV4TWT5,Atm Anthony Thomas Melillo,Stretch Pima Cotton Baby Tee,"Made From Atm’S Signature Peruvian Jersey, Thi...",Unknown,"True To Size. Xs=0, S=2-4, M=6-8, L=10."


In [9]:
data = pd.concat([data1, data2])
print(data.shape)
# Data clearning
data['brand_category'] = data['brand_category'].str.replace(r'(/|,|:)',' ')
data['brand_category'] = data['brand_category'].str.lower()
data['brand'] = data['brand'].str.title()
data['name'] = data['name'].str.title()
data['description'] = data['description'].str.title()
data['description'] = data['description'].str.replace('\n',' ')
data['details'] = data['details'].str.title()
data['details'] = data['details'].str.replace('\n',' ')
data.drop(labels = ['details'], inplace = True, axis = 1)
data = data.replace(np.nan, '', regex=True)
data = data.replace(r'(unknown|Unknown)', '', regex=True)
data.reset_index(inplace = True)
data = data.drop(labels = ['index'], axis = 1)
data

(48994, 6)


,product_id,brand,name,description,brand_category
0,01DSRPSZTDW2PGK1YWYXJGKZZ0,Fila,Original Fitness Sneakers,Vintage Fitness Leather Sneakers With Logo Pri...,themensstore shoes sneakers lowtop
1,01DSQXJBX0R7DCW7KTAC1SW547,Chanel,Hat,,
2,01DPGV8TGRAB993PF7Z3YWG2VR,Frame,Petit Oval Buckle Belt,A Timeless Leather Belt Crafted From Smooth Co...,accessories
3,01DSR8G3F7DBRTMP8THF97XSQ2,Lilly Pulitzer Kids,Little Gir'S & Girl'S Ariana One-Piece Upf 50+...,Pretty Ruffle Sleeves And Trim Elevate Essenti...,justkids girls214 girls swimwearcoverups justk...
4,01DSR8G5GP519DEDCSKBMWQVK5,Kissy Kissy,Baby Girl'S Endearing Elephants Pima Cotton Co...,Versatile Convertible Gown With Elephant Applique,justkids baby024months infantgirls footiesrompers
...,...,...,...,...,...
48989,01E2KZ045BCT1Q2Z2PBC0CWRGH,Atm Anthony Thomas Melillo,Micro Twill Pull On Pants,"These Casual Trousers Are Cut For An Easy, Dra...",
48990,01E2KXZ0WNQ96S64TDZE3WB7CX,Atm Anthony Thomas Melillo,Brushed Twill Crop Wide Leg Pants,Brushed Twill Gives These Cropped Wide-Leg Pan...,
48991,01E2KXWWY6JAGENED7P62090NP,Atm Anthony Thomas Melillo,Slim Crop Pants,"Tailored From A Soft, Dense Knit With Cropped,...",
48992,01E2KZTEPE7QK1V8THASF7HWNR,Atm Anthony Thomas Melillo,Camo Print Silk Skirt,Flatlock-Stitched Bias Seams Enhance The Fluid...,


In [10]:
# Function:
# Combine all inputs/columns into one string

def combine_col(input_data):
    '''Combine all inputs/columns into one string.
       Returns a DataFrame'''
    input_data['combined_data'] = input_data['name'] + " " + input_data['description'] + " " + input_data['brand_category']
    return input_data


In [11]:
# Function:
# Tokenize string and remove stopwords

def remove_stopwords(input_data):
    '''Tokenize string.
        Returns a list.'''
    
    # combine all columns - call combin_col() function
    data2 = combine_col(input_data)

    # remove_stopwords
    from nltk.corpus import stopwords
    import nltk
    import re
    
    regex_word_tokenize = nltk.RegexpTokenizer(r"(\w+['-]?[a-zA-Z']*[a-z]|[0-9]+-*[0-9]*)")
    nltk_stopwords = list((set(stopwords.words('english'))))
    
    # remove "unknown" since in the full_data dataset, some brand_category are unknown
    nltk_stopwords.append('unknown')

    result2 = []
    for line in data2['combined_data']:
        filtered_words = []
        if isinstance(line, str):
            line = re.sub(r'\d+\+*[\- ]*[\-]*',' ',line)
            for word in regex_word_tokenize.tokenize(line):
                if word.isdigit() == False:
                    if word.lower() not in nltk_stopwords:
                        filtered_words.append(word.lower())
            result2.append(" ".join(filtered_words))
        else:
            result2.append(np.nan)
    data2['rm_sw'] = result2
    return data2


In [12]:
def lemmatize_word(input_data):
    d1 = remove_stopwords(input_data)

    from nltk.stem import WordNetLemmatizer
    from nltk.corpus import stopwords
    nltk_stopwords = list((set(stopwords.words('english'))))
    regex_word_tokenize = nltk.RegexpTokenizer(r"(\w+['-]?[a-zA-Z']*[a-z]|[0-9]+-*[0-9]*)")
    lemmatizer = WordNetLemmatizer()

    result3 = []
    for i in range(len(d1['rm_sw'])):
        lemmatized = []
        if isinstance(d1['rm_sw'].iloc[i],str):
            for word in regex_word_tokenize.tokenize(d1['rm_sw'].iloc[i]):
                lemmatized.append(lemmatizer.lemmatize(word))
            result3.append(" ".join(lemmatized))
        else:
            result3.append(d1['rm_sw'].iloc[i])
    d1['lemmatized'] = result3
    d1['final'] = d1['lemmatized'] + " " + d1['brand']
    return d1

In [13]:
final = lemmatize_word(data)
final = pd.merge(left = final, right = attr, left_on = 'product_id', right_on = 'product_id', how = 'inner')

In [14]:
final.head()

,product_id,brand,name,description,brand_category,combined_data,rm_sw,lemmatized,final,product_color_id,attribute_name,attribute_value
0,01DTJCERF6F4NRZ2WSJFFA1EYS,Theory,Teah Stretch-Silk Camisole,"Beige Stretch-Silk Slips On 93% Silk, 7% Spand...",clothing tops tanks and camis,Teah Stretch-Silk Camisole Beige Stretch-Silk ...,teah stretch-silk camisole beige stretch-silk ...,teah stretch-silk camisole beige stretch-silk ...,teah stretch-silk camisole beige stretch-silk ...,01DTJCERGEHDMQ5FAJQS2PZ6NJ,neckline,scoop neck
1,01DTJCERF6F4NRZ2WSJFFA1EYS,Theory,Teah Stretch-Silk Camisole,"Beige Stretch-Silk Slips On 93% Silk, 7% Spand...",clothing tops tanks and camis,Teah Stretch-Silk Camisole Beige Stretch-Silk ...,teah stretch-silk camisole beige stretch-silk ...,teah stretch-silk camisole beige stretch-silk ...,teah stretch-silk camisole beige stretch-silk ...,01DTJCERGEHDMQ5FAJQS2PZ6NJ,sizing,regular
2,01DTJCERF6F4NRZ2WSJFFA1EYS,Theory,Teah Stretch-Silk Camisole,"Beige Stretch-Silk Slips On 93% Silk, 7% Spand...",clothing tops tanks and camis,Teah Stretch-Silk Camisole Beige Stretch-Silk ...,teah stretch-silk camisole beige stretch-silk ...,teah stretch-silk camisole beige stretch-silk ...,teah stretch-silk camisole beige stretch-silk ...,01DTJCERGEHDMQ5FAJQS2PZ6NJ,occasion,weekend
3,01DTJCERF6F4NRZ2WSJFFA1EYS,Theory,Teah Stretch-Silk Camisole,"Beige Stretch-Silk Slips On 93% Silk, 7% Spand...",clothing tops tanks and camis,Teah Stretch-Silk Camisole Beige Stretch-Silk ...,teah stretch-silk camisole beige stretch-silk ...,teah stretch-silk camisole beige stretch-silk ...,teah stretch-silk camisole beige stretch-silk ...,01DTJCERGEHDMQ5FAJQS2PZ6NJ,occasion,work
4,01DTJCERF6F4NRZ2WSJFFA1EYS,Theory,Teah Stretch-Silk Camisole,"Beige Stretch-Silk Slips On 93% Silk, 7% Spand...",clothing tops tanks and camis,Teah Stretch-Silk Camisole Beige Stretch-Silk ...,teah stretch-silk camisole beige stretch-silk ...,teah stretch-silk camisole beige stretch-silk ...,teah stretch-silk camisole beige stretch-silk ...,01DTJCERGEHDMQ5FAJQS2PZ6NJ,length_top,mid-length (at hips)


In [15]:
final.shape == pd.merge(left = data, right = attr, left_on = 'product_id', right_on = 'product_id', how = 'inner').shape

True

In [24]:
final.attribute_name.unique()

array(['neckline', 'sizing', 'occasion', 'length_top', 'fit', 'style',
       'material_clothing', 'gender', 'sleeve_length', 'category',
       'primary color', 'subcategory_top', 'dry_clean_only',
       'class_dress', 'subcategory_one_piece', 'length_one_piece',
       'strap', 'length_skirts', 'class_skirts', 'subcategory_bottom',
       'subcategory_shoe', 'upper_material', 'shoe_width',
       'additional color', 'pattern', 'class_booties', 'toe_style',
       'class_mules_and_slides', 'leg_style', 'length_pants_and_leggings',
       'trend', 'class_pants_and_leggings', 'wash', 'rise',
       'subcategory_sweater', 'class_sandals', 'heel_height',
       'heel_shape', 'class_pumps_and_heels', 'class_jumpsuit_and_romper',
       'embellishment', 'strap_material', 'subcategory_accessory',
       'closure_handbag', 'class_handbags', 'material_purse',
       'closure_top', 'subcategory_blazers_coats_and_jackets',
       'closure_blazers_coats_and_jackets',
       'length_blazers_coats

In [31]:
len(final.attribute_value[final['attribute_name']=='dry_clean_only'])

1526

In [17]:
# filter = ['color','category','occasion', 'embellishment']

# final  = pd.merge(left = input_data, right = attr, left_on = 'product_id', right_on = 'product_id')
# final[final.attribute_name.isin(filter)]

In [18]:
final.head()


,product_id,brand,name,description,brand_category,combined_data,rm_sw,lemmatized,final,product_color_id,attribute_name,attribute_value
0,01DTJCERF6F4NRZ2WSJFFA1EYS,Theory,Teah Stretch-Silk Camisole,"Beige Stretch-Silk Slips On 93% Silk, 7% Spand...",clothing tops tanks and camis,Teah Stretch-Silk Camisole Beige Stretch-Silk ...,teah stretch-silk camisole beige stretch-silk ...,teah stretch-silk camisole beige stretch-silk ...,teah stretch-silk camisole beige stretch-silk ...,01DTJCERGEHDMQ5FAJQS2PZ6NJ,neckline,scoop neck
1,01DTJCERF6F4NRZ2WSJFFA1EYS,Theory,Teah Stretch-Silk Camisole,"Beige Stretch-Silk Slips On 93% Silk, 7% Spand...",clothing tops tanks and camis,Teah Stretch-Silk Camisole Beige Stretch-Silk ...,teah stretch-silk camisole beige stretch-silk ...,teah stretch-silk camisole beige stretch-silk ...,teah stretch-silk camisole beige stretch-silk ...,01DTJCERGEHDMQ5FAJQS2PZ6NJ,sizing,regular
2,01DTJCERF6F4NRZ2WSJFFA1EYS,Theory,Teah Stretch-Silk Camisole,"Beige Stretch-Silk Slips On 93% Silk, 7% Spand...",clothing tops tanks and camis,Teah Stretch-Silk Camisole Beige Stretch-Silk ...,teah stretch-silk camisole beige stretch-silk ...,teah stretch-silk camisole beige stretch-silk ...,teah stretch-silk camisole beige stretch-silk ...,01DTJCERGEHDMQ5FAJQS2PZ6NJ,occasion,weekend
3,01DTJCERF6F4NRZ2WSJFFA1EYS,Theory,Teah Stretch-Silk Camisole,"Beige Stretch-Silk Slips On 93% Silk, 7% Spand...",clothing tops tanks and camis,Teah Stretch-Silk Camisole Beige Stretch-Silk ...,teah stretch-silk camisole beige stretch-silk ...,teah stretch-silk camisole beige stretch-silk ...,teah stretch-silk camisole beige stretch-silk ...,01DTJCERGEHDMQ5FAJQS2PZ6NJ,occasion,work
4,01DTJCERF6F4NRZ2WSJFFA1EYS,Theory,Teah Stretch-Silk Camisole,"Beige Stretch-Silk Slips On 93% Silk, 7% Spand...",clothing tops tanks and camis,Teah Stretch-Silk Camisole Beige Stretch-Silk ...,teah stretch-silk camisole beige stretch-silk ...,teah stretch-silk camisole beige stretch-silk ...,teah stretch-silk camisole beige stretch-silk ...,01DTJCERGEHDMQ5FAJQS2PZ6NJ,length_top,mid-length (at hips)


In [32]:
final.to_csv('full_data_joined_attr_processed.csv', index=False)

In [ ]:
# # N-Gram

# input_data = lemmatize_word(data)

# from sklearn.feature_extraction.text import CountVectorizer

# vectorizer = CountVectorizer(ngram_range=(2,2), min_df = 2)
# gram = vectorizer.fit_transform(input_data["result"])
# gram_features = pd.DataFrame(gram.toarray(), columns = vectorizer.get_feature_names())
# print(f'Shape of X_pos: {gram_features.shape}')


# from sklearn.feature_extraction.text import TfidfVectorizer

# # Kept the number of features at 5000 because my computer cannot handle more than 5000 (the kernel will die)
# tfidfvectorizer = TfidfVectorizer(ngram_range = (2,2), min_df = 2)
# corpus = list(input_data["result"].values)
# X = tfidfvectorizer.fit_transform(corpus)
# X_terms = tfidfvectorizer.get_feature_names()
# tf_idf = pd.DataFrame(X.toarray().transpose(), index=X_terms)
# tf_idf = tf_idf.sum(axis=1)
# score = pd.DataFrame(tf_idf, columns=["score"])
# score.sort_values(by="score", ascending=False, inplace=True)
# score

# # gram_features
len(d1['rm_sw'])
# regex_word_tokenize.tokenize(d1['rm_sw'].iloc[1])


In [ ]:
input_data = data
d1 = remove_stopwords(input_data)

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk_stopwords = list((set(stopwords.words('english'))))
regex_word_tokenize = nltk.RegexpTokenizer(r"(\w+['-]?[a-zA-Z']*[a-z]|[0-9]+-*[0-9]*)")
lemmatizer = WordNetLemmatizer()

result3 = []
for i in range(len(d1['rm_sw'])):
    lemmatized = []
    if isinstance(d1['rm_sw'].iloc[i],str):
        for word in regex_word_tokenize.tokenize(d1['rm_sw'].iloc[i]):
            lemmatized.append(lemmatizer.lemmatize(word))
        result3.append(" ".join(lemmatized))
    else:
        result3.append(d1['rm_sw'].iloc[i])
d1['lemmatized'] = result3
d1['final'] = d1['lemmatized'] + " " + d1['brand']
d1

In [ ]:
d1['rm_sw'][3]

In [ ]:
data = pd.concat([data1, data2])
print(data.shape)
# Data clearning
data['brand_category'] = data['brand_category'].str.replace(r'(/|,|:)',' ')
data['brand_category'] = data['brand_category'].str.lower()
data['brand'] = data['brand'].str.title()
data['name'] = data['name'].str.title()
data['description'] = data['description'].str.title()
data['description'] = data['description'].str.replace('\n',' ')
data.drop(labels = ['details'], inplace = True, axis = 1)
data = data.reset_index()
data

In [ ]:
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import stopwords
# nltk_stopwords = list((set(stopwords.words('english'))))
# lemmatizer = WordNetLemmatizer()
# lemmatized = []
# result3 = []
# for line in data.combined:
#     for word in regex_word_tokenize.tokenize(line):
#         lemmatized.append(lemmatizer.lemmatize(word))
#     result3.append(" ".join(lemmatized))
# result3

In [ ]:
# input_data = data
# result_df1 = pd.DataFrame(input_data['product_id'])
# result_df1['brand'] = input_data['brand']
# result_df1['combined_data'] = data.drop(labels = ['brand','product_id'], axis = 1).apply(' '.join, axis=1)



# # function remove stopword
# # combine all columns - call combin_col() function
# data2 = combine_col(input_data)

# # remove_stopwords
# from nltk.corpus import stopwords
# import re

# nltk_stopwords = list((set(stopwords.words('english'))))
# # remove "unknown" since in the full_data dataset, some brand_category are unknown
# nltk_stopwords.append('unknown')


# regex_word_tokenize = nltk.RegexpTokenizer(r"(\w+['-]?[a-zA-Z']*[a-z]|[0-9]+-*[0-9]*)")

# result2 = []

# for line in data2['combined_data']:
#     filtered_words = []
#     if isinstance(line, str):
#         line = re.sub(r'\d+\+*[\- ]*[\-]*',' ',line)
#         for word in regex_word_tokenize.tokenize(line):
#             if word.isdigit() == False:
#                 if word.lower() not in nltk_stopwords:
#                     filtered_words.append(word.lower())
#         result2.append(" ".join(filtered_words))
#     else:
#         result2.append(np.nan)
# result_df2 = result_df1[['product_id','brand']]
# result_df2['result'] = result2
# # result_df2['result'] = result_df2['result'] + " " + input_data['brand']




# # funtion lemmatize
# d1 = result_df2
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import stopwords
# nltk_stopwords = list((set(stopwords.words('english'))))
# lemmatizer = WordNetLemmatizer()

# result3 = []
# for line in d1['result']:
#     lemmatized = []
#     for word in regex_word_tokenize.tokenize(line):
#         lemmatized.append(lemmatizer.lemmatize(word))
#     result3.append(" ".join(lemmatized))
# result_df3 = result_df2[['product_id','brand']]
# result_df3['result'] = result3
# result_df3['result'] = result_df3['result'] + " " + result_df2['brand']
regex_word_tokenize = nltk.RegexpTokenizer(r"(\w+['-]?[a-zA-Z']*[a-z]|[0-9]+-*[0-9]*)")